# ROM-SHRED Tutorial on Kuramoto Sivashinsky

#### Import Libraries

In [71]:
# PYSHRED
%load_ext autoreload
%autoreload 2
from pyshred import ParametricDataManager, SHRED, SHREDEngine

# Other helper libraries
import matplotlib.pyplot as plt
from scipy.io import loadmat
import torch
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Load Kuramoto Sivashinsky dataset

In [72]:
import numpy as np
import urllib.request
# URL of the NPZ file
url = 'https://zenodo.org/records/14524524/files/KuramotoSivashinsky_data.npz?download=1'
# Local filename to save the downloaded file
filename = 'KuramotoSivashinsky_data.npz'
# Download the file from the URL
urllib.request.urlretrieve(url, filename)
# Load the data from the NPZ file
dataset = np.load(filename)

#### Initialize Data Manager

In [73]:
# Initialize ParametricSHREDDataManager
manager = ParametricDataManager(
    lags = 21,
    train_size = 0.8,
    val_size = 0.1,
    test_size = 0.1,
    )

#### Add datasets and sensors

In [74]:
data = dataset['u'] # shape (500, 201, 100)
# mu = dataset['mu'] # shape (500, 201, 2)

manager.add_data(
    data=data,
    # random=3,
    stationary=[(15,),(30,),(45,)],
    # params=mu,
    id = 'KS',
    compress = False
)

#### Analyze sensor summary

In [75]:
manager.sensor_measurements_df

,KS-0,KS-1,KS-2
0,0.154032,-1.140074,-0.006130
1,0.332515,-1.079287,-0.023609
2,0.361393,-1.070717,0.002758
3,0.404951,-1.049306,0.015024
4,0.485893,-1.031259,0.017951
...,...,...,...
100495,0.261712,-1.793277,-0.135204
100496,-0.622521,-1.301706,0.021596
100497,-1.591052,-0.838068,0.158474
100498,-2.129965,-0.466887,0.281114


In [76]:
manager.sensor_summary_df

,data id,sensor_number,type,loc/traj
0,KS,0,stationary,"(15,)"
1,KS,1,stationary,"(30,)"
2,KS,2,stationary,"(45,)"


#### Get train, validation, and test set

In [77]:
train_dataset, val_dataset, test_dataset= manager.prepare()

#### Initialize SHRED

When using a `ParametricDataManager`, ensure `latent_forecaster` is set to None.

In [78]:
shred = SHRED(sequence_model="LSTM", decoder_model="MLP", latent_forecaster=None)

#### Fit SHRED

In [79]:
val_errors = shred.fit(train_dataset=train_dataset, val_dataset=val_dataset, num_epochs=10, sindy_regularization=0)
print('val_errors:', val_errors)

Fitting SHRED...
Epoch 1: Average training loss = 0.024901
Validation MSE (epoch 1): 0.017020
Epoch 2: Average training loss = 0.011834
Validation MSE (epoch 2): 0.014274
Epoch 3: Average training loss = 0.009696
Validation MSE (epoch 3): 0.011126
Epoch 4: Average training loss = 0.008063
Validation MSE (epoch 4): 0.009599
Epoch 5: Average training loss = 0.007162
Validation MSE (epoch 5): 0.008390
Epoch 6: Average training loss = 0.006406
Validation MSE (epoch 6): 0.007549
Epoch 7: Average training loss = 0.005697
Validation MSE (epoch 7): 0.006496
Epoch 8: Average training loss = 0.005112
Validation MSE (epoch 8): 0.005727
Epoch 9: Average training loss = 0.004693
Validation MSE (epoch 9): 0.005416
Epoch 10: Average training loss = 0.004274
Validation MSE (epoch 10): 0.005388
val_errors: [0.01701978 0.01427372 0.01112562 0.00959897 0.00838996 0.00754887
 0.00649557 0.00572666 0.00541566 0.00538755]


#### Evaluate SHRED

In [80]:
train_mse = shred.evaluate(dataset=train_dataset)
val_mse = shred.evaluate(dataset=val_dataset)
test_mse = shred.evaluate(dataset=test_dataset)
print(f"Train MSE: {train_mse:.3f}")
print(f"Val   MSE: {val_mse:.3f}")
print(f"Test  MSE: {test_mse:.3f}")

Train MSE: 0.004
Val   MSE: 0.005
Test  MSE: 0.003


#### Initialize Parametric SHRED Engine for Downstream Tasks (TODO)

In [82]:
# engine = ParametricSHREDEngine(manager, shred)

#### Sensor Measurements to Latent Space

#### Decode Latent Space to Full-State Space